In [4]:
%load_ext autoreload
%autoreload 2

import json 
import pysftp
import os
import pandas_gbq
import pandas as pd
from modules.buckets import *
from modules.reproducibility import *
import logging

#Configure loggging
logging.basicConfig(filename='BigQuery.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.info('\n\n-------------New Big Query Logging Instance')

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable in order to interact, import the SFTP password from the same file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'powerschool-420113-db919282054b.json'
with open('powerschool-420113-db919282054b.json') as json_file:
    j = json.load(json_file) 
    sftp_pass = j['sftp_password']
    
# ----------------------------------------------------------
#Need to make this portion to where it assesses all files in the dir recursively. 
def main(SFTP_folder_name):

    instance = Create(
                project_id='powerschool-420113',
                location = 'us-south1',
                bucket=f'{SFTP_folder_name}bucket-iotaschools-1',
                local_dir = f'SFTP_folders\\{SFTP_folder_name}',
                db = SFTP_folder_name,
                table_name = 'ps_students',
                append_or_replace='replace',
                )

    SFTP_conn(sftp_pass, SFTP_folder_name) #Replicate SFTP folder onto local dir
    instance.process()#Replicate local dir into Buckets, pass dir filesinto Bucket & finally into Big Query tables


main('powerschool')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Dir contents:  ['Student_Records.csv', 'Students_export.txt']


Bucket powerschoolbucket-iotaschools-1 already exists.
File Students_export.txt in bucket powerschoolbucket-iotaschools-1 was overwritten by local file SFTP_folders\powerschool\Students_export.txt.
File Student_Records.csv in bucket powerschoolbucket-iotaschools-1 was overwritten by local file SFTP_folders\powerschool\Student_Records.csv.
Table powerschool-420113.powerschool.Student_Records already exists, argument is being called to REPLACE new data with incoming data


100%|██████████| 1/1 [00:00<?, ?it/s]


Table powerschool-420113.powerschool.Students_export already exists, argument is being called to REPLACE new data with incoming data


100%|██████████| 1/1 [00:00<00:00, 997.69it/s]
